**Key Learnings :**

1. Only one row filter is possible per table.
2. An alter table set filter for a second filter will override the first filter.

In [0]:
%sql
use dpaas_uccatalog_stg.adxdemo;
DROP TABLE IF EXISTS cities;
CREATE TABLE cities (city_name STRING, country_name STRING);
INSERT INTO cities VALUES ('New York','US'),('Bombay','India'), ('Shanghai','China'), ('Madras',null);

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
SELECT * FROM adxdemo.cities

city_name,country_name
New York,US
Bombay,India
Shanghai,China
Madras,null


Unpacking **RETURN IF (condition,expr1,expr2)** as a ternary operator.
 
 if **_condition_** is true against each row, then **_expr1_** is expressed 
 if **_condition_** is false then the **_expr2_** is expressed. expr1, expr2 can both be a statements as, eg. column _country_name = 'India'_ or _country_name = 'US'_.
 As in RETURN IF (condition,**_country_name = 'India'_**, **_country_name = 'US'_**)
 
 Boolean values for **_expr1_** or **_expr2_**
 when **_condition_** is true and **_expr1_** is marked as true then rows of tables that pass the condition are pushed to the user, if **_expr1_** is marked as false then rows of tables are not pushed to the user. As in RETURN IF (condition,**_true_**,expr2) or RETURN IF (condition,**_false_**,expr2)

 Deduplication -
 if both **_expr1_** and **_expr2_** push out duplicate values to the user then the records are deduplicated before displaying



In [0]:
%sql
use dpaas_uccatalog_stg.adxdemo;
CREATE OR REPLACE FUNCTION country_filter(country_name STRING)
RETURN IF(isnull(country_name), true, true);


In [0]:
%sql
ALTER TABLE cities SET ROW FILTER country_filter ON (country_name);

In [0]:
%sql
select * from dpaas_uccatalog_stg.adxdemo.cities;

city_name,country_name
New York,US
Bombay,India
Shanghai,China
Madras,null


In [0]:
%sql
ALTER TABLE dpaas_uccatalog_stg.adxdemo.cities DROP ROW FILTER;

In [0]:
%sql
SELECT * FROM adxdemo.cities

city_name,country_name
New York,US
Bombay,India
Shanghai,China
Madras,null


In [0]:
%sql
SELECT IS_ACCOUNT_GROUP_MEMBER('admin');

is_account_group_member(admin)
false


In [0]:
%sql
SHOW GROUPS WITH USER `tat18156@adobe.com`;

name,directGroup


In [0]:
%sql
SET RULE show_rows_by_country 
ON CATALOG dpaas_uccatalog_stg 
ROW FILTER country_filter
TO admins
FOR TABLES
WHEN COLUMNS col_has_tag_value('country');

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-358852789270704>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', "SET RULE show_rows_by_country \nON CATALOG dpaas_uccatalog_stg \nROW FILTER country_filter\nTO admins\nFOR TABLES\nWHEN COLUMNS col_has_tag_value('country');\n")

File /databricks/python/lib/python3.10/site-packages/IPython/core/interactiveshell.py:2478, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2476 with self.builtin_trap:
   2477     args = (magic_arg_s, cell)
-> 2478     result = fn(*args, **kwargs)
   2480 # The code below prevents the output from being displayed
   2481 # when using magics with decodator @output_can_be_silenced
   2482 # when the last Python token in the expression is a ';'.
   2483 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/dbruntime/sql_magic/sql_magic.py